In [1]:
from dataclasses import dataclass

In [2]:
@dataclass
class TimedElement():
    start_time: int
    end_time: int
    text: str

class CTMLine(TimedElement):
    pass

class RDAPI(TimedElement):
    pass

In [13]:
aa = [
    CTMLine(0, 5, "foo"),
    CTMLine(5, 10, "foo"),
    CTMLine(10, 15, "foo"),
    CTMLine(21, 25, "foo"),
    CTMLine(30, 35, "foo"),
    CTMLine(41, 45, "foo"),
    CTMLine(50, 55, "foo"),
    CTMLine(50, 65, "foo"),
]

In [4]:
ab = [
    RDAPI(20, 35, "blah"),
    RDAPI(40, 55, "blah"),
]

In [16]:
from typing import List

@dataclass
class FilteredPair():
    ctmlines: List[CTMLine]
    riksdag_segments: List[RDAPI]
    speaker_name: str = ""

In [5]:
from copy import deepcopy

In [ ]:
within = False
start = True

last_i = i = last_j = j = 0
pairs = []


while i < len(aa) - 1:
    if start or not within:
        sought = ab[j].start_time
    else:
        sought = ab[j].end_time
    while j < len(ab) - 1:
        if aa[i].end_time < sought:
            i += 1
        else:
            if start:
                start = False
                within = True
            if within:
                rd = ab[j]
                spkr = ab[j].text
                within = False
            else:
                rd = None
                spkr = ""
                within = True
            pairs.append(FilteredPair(deepcopy(aa[last_i:i]), rd, spkr))
            i += 1
            last_i = i
            j += 1
            


In [34]:
pairs[1]

IndexError: ignored